### 测试LLM的多轮对话能力

我们所看到的大语言模型，当调用它实现问答的时候，实际上是无状态的，模型本身不会记得你之前问的问题以及回答的内容。

但是，我们也有变通的方式，就是将历史的聊天内容，以上下文的方式，和问题一起发送给模型来生成答案，模型自然就知道了之前的聊天内容。为了实现这样的多轮对话，最简单的方式就是我们自己将历史聊天内容进行拼接即可。但是，LangChain 以及帮我们实现好了。

In [ ]:
# %pip install langchain==0.0.270

In [20]:
from langchain import __version__
print(__version__)

0.0.270


In [21]:
# 查看现有的部署在sagemaker上的endpoints
endpoints = !aws sagemaker list-endpoints
endpoints = json.loads("".join(endpoints))
for ep in endpoints['Endpoints']:
    print(ep['EndpointName'])

mt-chatglm2-g4dn
mt-cpmbee-g4dn
mt-chatglm2-6b-fastllm-fp16-g4dn
mt-cn-alpaca2-7b-g5
mt-baichuan-g5
mt-chatglm2-6b-ft-g4dn
mt-sd-2-djl-inf2


首先我们使用 LangChain 框架，生成一个 *SagemakerEndpoint* 类型的 model 对象：

In [ ]:
# from prompt_generator import generate_prompt as gen_prompt
import prompt_generator as pg
import importlib

importlib.reload(pg)

In [60]:
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.WARN)

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input = {"inputs": prompt, "parameters": {"temperature": 0.01}}
        logging.info("prompt: %s", prompt)
        input_str = json.dumps(input)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        logger.info("response_json: %s", response_json)
        # return response_json[0]["generated_text"] # for THI inferencer
        return response_json["outputs"]

content_handler = ContentHandler()

In [61]:
sagemaker_endpoint_name = "mt-chatglm2-g4dn"

chatglm_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    content_handler=content_handler
)

在 LangChain 中，帮我们实现了各种 Memory 对象，用于对聊天历史记录进行管理。然后，在创建 Chain 的时候，只要传递这个 memory 对象，我们的问答机器人就具备了记忆能力。

In [62]:
# from prompt_generator import generate_prompt as gen_prompt
import prompt_generator as pg
import importlib

importlib.reload(pg)


<module 'prompt_generator' from '/home/ec2-user/SageMaker/LLM_Workshop/prompt_generator.py'>

In [63]:
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain

template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
conversation_memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
conversation_chain = LLMChain(
    llm=chatglm_model,
    memory=conversation_memory,
    prompt=prompt
)

In [64]:
# 开始前清除memory
conversation_memory.clear()

In [70]:
hist_str = conversation_memory.load_memory_variables({})['chat_history']

In [71]:
hist_str.split('\n')

['Human: 你会做数学题吗？',
 'AI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？',
 'Human: 3+4+5等于多少',
 'AI: 3+4+5=12',
 'Human: 再加5呢？',
 'AI: 再加5等于17。']

In [66]:
conversation_chain("你会做数学题吗？")

{'human_input': '你会做数学题吗？',
 'chat_history': '',
 'text': '是的，我可以做数学题。请问您有什么数学问题需要帮助吗？'}

In [67]:
conversation_chain("3+4+5等于多少")

{'human_input': '3+4+5等于多少',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？',
 'text': '3+4+5=12'}

In [68]:
conversation_chain("再加5呢？")

{'human_input': '再加5呢？',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？\nHuman: 3+4+5等于多少\nAI: 3+4+5=12',
 'text': '再加5等于17。'}

In [10]:
conversation_chain("再乘以2呢？")

{'human_input': '再乘以2呢？',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？\nHuman: 3+4+5等于多少\nAI: 3+4+5=12\nHuman: 再加5呢？\nAI: 再加5等于17。',
 'text': '再乘以2等于34。'}

In [12]:
conversation_chain("再减2呢")

{'human_input': '再减2呢',
 'chat_history': 'Human: 3+4+5等于多少\nAI: 3+4+5=12\nHuman: 再加5呢？\nAI: 再加5等于17。\nHuman: 再乘以2呢？\nAI: 再乘以2等于34。',
 'text': '再减2等于32。'}

#### 测试简单的聊天场景

In [11]:
# 开始前清除memory
conversation_memory.clear()

In [12]:
conversation_chain("我明天去跟暗恋很久的女神吃饭，我应该穿什么衣服？")

{'human_input': '我明天去跟暗恋很久的女神吃饭，我应该穿什么衣服？',
 'chat_history': '',
 'text': '穿什么衣服当然要根据场合和餐厅的要求来选择。一般来说,餐厅会要求比较正式的服装,比如穿西装或礼服。如果餐厅没有特别的要求,你可以选择穿一些比较舒适的衣服,比如牛仔裤或休闲裙。不过,考虑到你是去跟暗恋很久的女神吃饭,最好还是选一些比较正式的衣服,这样会显得更加尊重和恰当。'}

In [13]:
conversation_chain("可是我没有这样的衣服，有必要买吗？")

{'human_input': '可是我没有这样的衣服，有必要买吗？',
 'chat_history': 'Human: 我明天去跟暗恋很久的女神吃饭，我应该穿什么衣服？\nAI: 穿什么衣服当然要根据场合和餐厅的要求来选择。一般来说,餐厅会要求比较正式的服装,比如穿西装或礼服。如果餐厅没有特别的要求,你可以选择穿一些比较舒适的衣服,比如牛仔裤或休闲裙。不过,考虑到你是去跟暗恋很久的女神吃饭,最好还是选一些比较正式的衣服,这样会显得更加尊重和恰当。',
 'text': '这取决于场合和重要性。如果这个餐厅非常正式,或者你是去参加一个重要的活动或会议,那么穿正装是必要的。否则,穿着舒适和得体也是可以的。如果你真的没有合适的衣服,也不必非得买一件新的。你可以考虑租一套礼服或者借一件合适的衣服。不过,最重要的是在吃饭时保持整洁和得体,不要穿太暴露或太随意。'}

In [14]:
conversation_chain("那头发呢？")

{'human_input': '那头发呢？',
 'chat_history': 'Human: 我明天去跟暗恋很久的女神吃饭，我应该穿什么衣服？\nAI: 穿什么衣服当然要根据场合和餐厅的要求来选择。一般来说,餐厅会要求比较正式的服装,比如穿西装或礼服。如果餐厅没有特别的要求,你可以选择穿一些比较舒适的衣服,比如牛仔裤或休闲裙。不过,考虑到你是去跟暗恋很久的女神吃饭,最好还是选一些比较正式的衣服,这样会显得更加尊重和恰当。\nHuman: 可是我没有这样的衣服，有必要买吗？\nAI: 这取决于场合和重要性。如果这个餐厅非常正式,或者你是去参加一个重要的活动或会议,那么穿正装是必要的。否则,穿着舒适和得体也是可以的。如果你真的没有合适的衣服,也不必非得买一件新的。你可以考虑租一套礼服或者借一件合适的衣服。不过,最重要的是在吃饭时保持整洁和得体,不要穿太暴露或太随意。',
 'text': '头发的穿着也很重要,尤其是在正式场合。如果你打算穿正装,建议选择一些简单大方的发型,比如经典的马尾或盘发。如果你选择穿休闲装,可以选择一些比较随意的发型,比如束发或披散。不过,无论你选择什么样的发型,都要确保它们干净整洁,不要油腻或混乱。'}

#### 不同的memory类型

刚才使用的 Memory 类型是 ConversationBufferWindowMemory，它是基于内存的、有固定窗口大小的对象。我们创建的窗口大小为3，如果对话轮数超过3轮，就会将最老的一个对话记录清除。可以从参数的 “chat_history” 看到历史记录。

很显然，这种简单粗暴的方式有时候不适用。还好 LangChain 给我们提供了多种类型的 Memory。下面来看看几种常用的 Memory：
 * ConversationBufferMemory: 将历史记录保存在Buffer中，如果对话多轮，则有可能会超出buffer大小。
 * ConversationEntityMemory: 将对话记录中的entity实例提取出来保存起来，在后续的对话中，会根据实体以及它们之间的关系作为上下文，回答用户的问题。
 * Conversation Knowledge Graph Memory: 使用基于内存的知识库的形式，从用户输入中提取三元组，并保存，用于后续的问答。
 * ConversationSummaryMemory: 如果不指定对话轮数，而一味的将对话内容保存在memory中的话，很快就会因为超出token限制而出错。如果使用了window memory，则会丢弃老的对话记录。而使用这个ConversationSummaryMemory，他就会根据聊天历史生成summery，再将这个summery信息放到对话上下文中，用于后续对话。这样即使是很多轮对话，也不会超出token显示或丢失重要信息。
 * VectorStoreRetrieverMemory: 使用这个memory，他会将聊天历史保存在向量数据库中，然后每次对话的时候，根据用户输入查询最相近的几条记录，并作为上下文，回答用户问题。跟这个类似，也有很多将记录保存到各种数据库的memory。